<a href="https://colab.research.google.com/github/R0b0t-Maker/LLM-T/blob/main/Assistant_v001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pymupdf pandas transformers


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Extract text from PDFs**

In [ ]:
import fitz  # PyMuPDF
import pandas as pd

# Extract text from the template PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    pdf_document = fitz.open(pdf_path)
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    return text

template_text = extract_text_from_pdf("/content/drive/MyDrive/Colab Notebooks/Assistant/template.pdf")
information_text = extract_text_from_pdf("/content/drive/MyDrive/Colab Notebooks/Assistant/information.pdf")


**Identify empty cells in the template PDF**

In [ ]:
import re

def find_empty_cells(template_text):
    return re.findall(r'_____', template_text)

empty_cells = find_empty_cells(template_text)


**Extract relevant information from the information PDF**

In [ ]:
from transformers import pipeline

# Load a pre-trained model for question answering
qa_pipeline = pipeline("question-answering")

# Function to find answers to the questions
def find_answers(questions, context):
    answers = []
    for question in questions:
        result = qa_pipeline(question=question, context=context)
        answers.append(result['answer'])
    return answers

# Define questions based on the template's context
questions = [
    "What is the minimum base material temperature?",
    "What is the concrete class?",
    "What is the concrete category?",
    # Add more questions based on the template's context
]

answers = find_answers(questions, information_text)


No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recom

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

**Fill the template with extracted information**

In [ ]:
def fill_template(template_text, answers):
    filled_template = template_text
    for answer in answers:
        filled_template = filled_template.replace("_____", answer, 1)
    return filled_template

filled_template_text = fill_template(template_text, answers)

# Save the filled template
with open("filled_template.txt", "w") as file:
    file.write(filled_template_text)


**Convert filled text back to PDF**

In [ ]:
from fpdf import FPDF

class PDF(FPDF):
    def __init__(self):
        super().__init__()

    def header(self):
        self.set_font("Arial", size=12)
        self.cell(0, 10, 'Filled Template', ln=True, align='C')

    def chapter_title(self, title):
        self.set_font("Arial", 'B', 12)
        self.cell(0, 10, title, ln=True)
        self.ln(10)

    def chapter_body(self, body):
        self.set_font("Arial", size=12)
        self.multi_cell(0, 10, body)
        self.ln()

pdf = PDF()
pdf.add_page()
pdf.chapter_title('Filled Template')
pdf.chapter_body(filled_template_text)
pdf.output('filled_template.pdf')
